In [17]:
import os
import numpy as np
import pandas as pd
from datetime import date, timedelta
from sklearn import metrics
# from sklearn.preprocessing import MinMaxScaler, LabelEncoder
import tensorflow as tf
import keras
from keras.models import Sequential, Model
from keras.layers import *
from keras import optimizers
import gc

from Utils import *
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # suppress tf warnings

timesteps = 365

df, promo_df, items, stores = load_unstack('pw')

# data after 2015
df = df[pd.date_range(date(2013,1,1), date(2016,8,15))]
promo_df = promo_df[pd.date_range(date(2013,1,1), date(2016,8,15))]

promo_df = promo_df[df[pd.date_range(date(2013,1,1), date(2016,8,15))].max(axis=1)>0]
df = df[df[pd.date_range(date(2013,1,1), date(2016,8,15))].max(axis=1)>0]
promo_df = promo_df.astype('int')

#print("df top 5", df[:5])
#print("promo_df top 5", promo_df[:5])

df_test = pd.read_csv("../../../data/favgrocery/test.csv", usecols=[0, 1, 2, 3, 4], dtype={'onpromotion': bool},
                      parse_dates=["date"]).set_index(['store_nbr', 'item_nbr', 'date'])
item_nbr_test = df_test.index.get_level_values(1)
item_nbr_train = df.index.get_level_values(1)
item_inter = list(set(item_nbr_train).intersection(set(item_nbr_test)))
df = df.loc[df.index.get_level_values(1).isin(item_inter)]
promo_df = promo_df.loc[promo_df.index.get_level_values(1).isin(item_inter)]

df_index = df.index
del item_nbr_test, item_nbr_train, item_inter, df_test; gc.collect()

#print("df top 5 before gener train:", df[:5])
#print("promo_df top 5 before gener train:", promo_df[:5])
#print("items top 5 before gener train:", items[:5])
#print("stores top 5 before gener train:", stores[:5])

train_data = train_generator(df, promo_df, items, stores, timesteps, date(2016, 7, 9),
                                           n_range=380, day_skip=1, batch_size=1000, aux_as_tensor=True, reshape_output=2)
Xval, Yval = create_dataset(df, promo_df, items, stores, timesteps, date(2016, 7, 26),
                                     aux_as_tensor=True, reshape_output=2)
print(len(Xval))
print(Xval[1].shape)
print(len(Yval))
print(Yval[0].shape)

Xtest, _ = create_dataset(df, promo_df, items, stores, timesteps, date(2016, 2, 16),
                                    aux_as_tensor=True, is_train=False, reshape_output=2)

10
(11, 365, 1)
11
(16,)


In [20]:
w = (Xval[7][:, 0, 2] * 0.25 + 1) / (Xval[7][:, 0, 2] * 0.25 + 1).mean()
latent_dim = 100

seq_in = Input(shape=(timesteps, 1))
is0_in = Input(shape=(timesteps, 1))
print(seq_in)
print(is0_in)

promo_in = Input(shape=(timesteps+16, 1))
yearAgo_in = Input(shape=(timesteps+16, 1))
quarterAgo_in = Input(shape=(timesteps+16, 1))
item_mean_in = Input(shape=(timesteps, 1))
store_mean_in = Input(shape=(timesteps, 1))
# store_family_mean_in = Input(shape=(timesteps, 1))
weekday_in = Input(shape=(timesteps+16,), dtype='uint8')
weekday_embed_encode = Embedding(7, 4, input_length=timesteps+16)(weekday_in)
# weekday_embed_decode = Embedding(7, 4, input_length=timesteps+16)(weekday_in)
dom_in = Input(shape=(timesteps+16,), dtype='uint8')
dom_embed_encode = Embedding(31, 4, input_length=timesteps+16)(dom_in)
print(weekday_in)
print(weekday_embed_encode)


Tensor("input_5:0", shape=(?, 365, 1), dtype=float32)
Tensor("input_6:0", shape=(?, 365, 1), dtype=float32)
Tensor("input_12:0", shape=(?, 381), dtype=uint8)
Tensor("embedding_1/embedding_lookup/Identity:0", shape=(?, 381, 4), dtype=float32)


In [21]:
cat_features = Input(shape=(timesteps+16, 6))

item_family = Lambda(lambda x: x[:, :, 0])(cat_features)
item_class = Lambda(lambda x: x[:, :, 1])(cat_features)
item_perish = Lambda(lambda x: x[:, :, 2])(cat_features)
store_nbr = Lambda(lambda x: x[:, :, 3])(cat_features)
store_cluster = Lambda(lambda x: x[:, :, 4])(cat_features)
store_type = Lambda(lambda x: x[:, :, 5])(cat_features)

print(cat_features)
print(item_family)

Tensor("input_14:0", shape=(?, 381, 6), dtype=float32)
Tensor("lambda_1/strided_slice:0", shape=(?, 381), dtype=float32)


In [22]:
family_embed = Embedding(33, 8, input_length=timesteps+16)(item_family)
class_embed = Embedding(337, 8, input_length=timesteps+16)(item_class)
store_embed = Embedding(54, 8, input_length=timesteps+16)(store_nbr)
cluster_embed = Embedding(17, 3, input_length=timesteps+16)(store_cluster)
type_embed = Embedding(5, 2, input_length=timesteps+16)(store_type)

print(family_embed)


Tensor("embedding_3/embedding_lookup/Identity:0", shape=(?, 381, 8), dtype=float32)


In [23]:
# Encoder
encode_slice = Lambda(lambda x: x[:, :timesteps, :])
encode_features = concatenate([promo_in, yearAgo_in, quarterAgo_in, weekday_embed_encode,
                               family_embed, Reshape((timesteps+16,1))(item_perish), store_embed, cluster_embed, type_embed], axis=2)
encode_features = encode_slice(encode_features)
print(encode_features)

Tensor("lambda_7/strided_slice:0", shape=(?, 365, 29), dtype=float32)
